In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 데이터 로딩
data = pd.read_csv('C:\\Users\\2004s\\Downloads\\archive\\GOOGL.csv', index_col='Date', parse_dates=True)

# 필요한 피처 선택
features = ['Open', 'High', 'Low', 'Close(t)', 'Volume']
data = data[features]

# 데이터 전처리: 가격 스케일링
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# 데이터 나누기 (학습과 테스트)
train_size = int(len(data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# 학습 데이터 생성 (time_step 만큼의 시점에 대한 데이터)
time_step = 60  # 과거 60일을 사용하여 예측
def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step)])
        Y.append(dataset[i + time_step, 3])  # Close price 예측
    return np.array(X), np.array(Y)

X_train, Y_train = create_dataset(train_data, time_step)
X_test, Y_test = create_dataset(test_data, time_step)

# LSTM 모델 정의
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))  # 하나의 출력을 위한 Dense 레이어

model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, Y_train, epochs=20, batch_size=32)

# 예측
predicted_price = model.predict(X_test)

# 예측 결과 역변환 (가격 스케일 복원)
predicted_price_full = np.hstack((np.zeros((predicted_price.shape[0], 4)), predicted_price))
predicted_price = scaler.inverse_transform(predicted_price_full)[:, -1]

# 실제 가격 역변환
Y_test_full = np.hstack((np.zeros((Y_test.shape[0], 4)), Y_test.reshape(-1, 1)))
Y_test_actual = scaler.inverse_transform(Y_test_full)

# ---- 마지막 일주일만 표시하는 그래프 ----
plt.figure(figsize=(14, 7))

# 마지막 일주일의 실제 가격 데이터 가져오기
last_week_actual = Y_test_actual[-7:, -1]  # 실제 가격 (Close 값)
last_week_dates = data.index[-7:]         # 마지막 7일의 날짜 인덱스

# 마지막 일주일의 예측 가격 가져오기
last_week_predicted = predicted_price[-7:]  # 예측 가격 (Close 값)

# 실제 가격 그래프
plt.plot(last_week_dates, last_week_actual, color='blue', label='Actual Prices (Last Week)')

# 예측 가격 그래프
plt.plot(last_week_dates, last_week_predicted, color='orange', label='Predicted Prices (Last Week)')

# 다음날 예측 가격
next_day_pred = model.predict(X_test[-1].reshape(1, time_step, len(features)))  # 마지막 테스트 샘플의 예측

# next_day_pred에 맞는 형태로 확장: 5개의 피처를 가진 배열로 만들어줍니다.
next_day_pred_full = np.hstack((np.zeros((next_day_pred.shape[0], 4)), next_day_pred))

# 예측 결과 역변환 (스케일 복원)
next_day_pred_full = scaler.inverse_transform(next_day_pred_full)

# 다음날 예측 가격만 추출
next_day_price = next_day_pred_full[0, -1]  # Close 값

# 예측된 다음날 가격의 날짜 설정
next_day_index = last_week_dates[-1] + pd.Timedelta(days=1)  # 마지막 날짜의 다음날

# 다음날 예측 가격을 그래프에 점으로 표시
plt.scatter(next_day_index, next_day_price, color='red', marker='x', s=100, label='Next Day Predicted Price')

# 다음날 예측 가격 주석 추가
plt.annotate(f'{next_day_price:.2f}', xy=(next_day_index, next_day_price), 
             xytext=(next_day_index, next_day_price + 0.02e7),
             arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=10, color='red')

# 제목, 라벨 설정
plt.title('LSTM - Last Week Prices and Next Day Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()

# 그래프 출력
plt.show()
